In [ ]:
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from instance_selection.operator.metrics import calculate_gmean_mauc
from instance_selection.parameter.parameter import *
import machine_learning.samme.samme as SAMME
import scipy.io as sio  # 从.mat文件中读取数据集

DATASETS=[Satellite,Contraceptive,Car,Balance_Scale,Dermatology,Chess,Nursery,WallRobot,Ecoli,Glass] # 数据集名称（包含对应参数的字典形式）
for DATASET in DATASETS:  
    datasetname = DATASET['DATASETNAME'].split('.')[0]
    mat_data = sio.loadmat(IMBALANCED_DATASET_PATH + DATASET['DATASETNAME'])  # 加载、划分数据集
    x = mat_data['X']
    y = mat_data['Y'][:, 0]  # mat_data['Y']得到的形状为[n,1]，通过[:,0]，得到形状[n,]
    x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=RANDOM_SEED)  # 划分数据集
    scaler = StandardScaler()  # 数据的标准化
    x_train = scaler.fit_transform(x_train)
    x_test = scaler.transform(x_test)
    model = MLPClassifier(hidden_layer_sizes=(DATASET['HIDDEN_SIZE'],), max_iter=DATASET['MAX_ITER'],
                          random_state=RANDOM_SEED, learning_rate_init=DATASET['LEARNING_RATE'])
    gmean_results = []
    mauc_results = []
    num_runs = 30
    for i in range(num_runs):
        samme = SAMME(POPSIZE, model, x_train, y_train)
        samme.train()
        y_pred_prob = samme.predict_prob(x_test)
        gmean, mauc, recall_per_class = calculate_gmean_mauc(y_pred_prob, y_test)

        print(f"第{i + 1}次执行，G-Mean: {gmean},mAUC: {mauc},Recall: {recall_per_class}")
        gmean_results.append(gmean)
        mauc_results.append(mauc)
    print(f"Avg-G-Mean: {np.mean(gmean_results, axis=0)}")
    print(f"Avg-mAUC: {np.mean(mauc_results, axis=0)}")
